First we must import numpy and SINDy

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pysindy import SINDy
from pysindy.feature_library import *
from pysindy.differentiation import *
# from pysindy.optimizers import *
from scipy.integrate import odeint, solve_ivp

file = '../ReducedTorqueData.csv'
pd.read_csv(file, nrows=3)

,timeInS,i_aInA,i_bInA,i_cInA,u_aInV,u_bInV,u_cInV,epsilon_elInRad
0,0.000000,1.765625,3.453125,0.937500,-0.322917,-0.677083,1.000000,-2.403804
1,0.000001,1.578125,3.421875,0.859375,-0.177083,-0.750000,0.927083,-2.403765
2,0.000002,1.578125,3.562500,0.750000,-0.114583,-0.854167,0.968750,-2.403726


In [3]:
data = np.loadtxt(file, delimiter=',', skiprows=1)    # Faster method

# Define State Vectors
t = data[:, 0]
X = data[:, 1:]

In [4]:
# Given Parameters
n_me = 16.67    # Hz
U_DC = 294      # Volts DC
Rs = 18e-3      # Ohms
Ld = 370e-6     # Henry
Lq = 1200e-6    # Henry
psi_p = 66e-3   # Vs
i_dqmax = 240   # Amps
p = 3           # pole pair number

In [5]:
# Calculate Torque from variables
i_abc = X[:, :3]
u_abc = X[:, 3:6]
e = X[:, 6]
i_dq = np.zeros((len(X), 2))
u_dq = np.zeros((len(X), 2))
for i in range(len(data)):
    trig = np.array([[np.cos(e[i]), -np.cos(e[i] + np.pi/3), -np.cos(e[i] - np.pi/3)],
                    [-np.sin(e[i]), np.sin(e[i] + np.pi/3), np.sin(e[i] - np.pi/3)]])
    i_dq[i] = np.matmul(trig, i_abc[i])
    u_dq[i] = np.matmul(trig, u_abc[i])

In [6]:
psi_d = Ld*i_dq[:, 0] + psi_p
psi_q = Lq*i_dq[:, 1]
T = 1.5*p*(psi_d*i_dq[:, 1] - psi_q*i_dq[:, 0])

In [12]:
# Instantiate model
model = SINDy(
              feature_library=PolynomialLibrary(degree=1),
              # differentiation_method=SmoothedFiniteDifference(),
              feature_names=['i_a','i_b','i_c','u_a','u_b','u_c','E_el']
            )

model.fit(X, t=t)
model.print()
print(model.coefficients())
model.score(X, t=t[1]-t[0])

(i_a)' = -61610.850 1 + 10175.706 i_a + 11319.365 i_b + 10194.894 i_c + -779338474.617 u_a + -779338478.393 u_b + -779338954.129 u_c + 477.196 E_el
(i_b)' = 30419.605 1 + -5896.876 i_a + -4369.941 i_b + -4312.699 i_c + 1163986222.376 u_a + 1163986632.108 u_b + 1163986411.945 u_c + 1199.228 E_el
(i_c)' = 35443.478 1 + -6509.856 i_a + -6206.872 i_b + -5162.023 i_c + 1217631282.888 u_a + 1217630890.294 u_b + 1217631679.847 u_c + 755.148 E_el
(u_a)' = 23713.178 1 + -6622.137 i_a + 985.205 i_b + -2508.623 i_c + 2773380297.536 u_a + 2773381067.075 u_b + 2773379862.815 u_c + 5129.519 E_el
(u_b)' = -19502.697 1 + -1019.142 i_a + 2595.554 i_b + 9705.235 i_c + 12832393.662 u_a + 12832909.161 u_b + 12833996.325 u_c + 4295.805 E_el
(u_c)' = -4210.457 1 + 7641.276 i_a + -3580.764 i_b + -7196.616 i_c + -2786212685.314 u_a + -2786213970.352 u_b + -2786213853.255 u_c + -9425.325 E_el
(E_el)' = 2.321 1 + -7.268 i_a + -13.085 i_b + 17.901 i_c + 5487137.722 u_a + 5487137.053 u_b + 5487138.449 u_c + 4.505

0.000775393900224884

In [8]:
tspan = [t[0], t[-1]]
x0 = X[0]
# x_SINDy = solve_ivp(, tspan, x0, t_eval=t)

In [16]:
sim = model.simulate(x0, t=t)       # sim should match X if there was no error
print(sim)

capi_return is NULL
Call-back cb_f_in_lsoda__user__routines failed.


KeyboardInterrupt: 

In [ ]:
TorqueSINDy = pass

In [15]:
# plt.figure(figsize=(12,5))
# # plt.plot(t, T)
# # plt.plot(t, X)
# plt.plot(sim[:, 0], sim[:, 1])
# plt.xlabel('t')
# plt.ylabel('Torque')
# plt.legend()
# plt.show()

capi_return is NULL
Call-back cb_f_in_lsoda__user__routines failed.


ValueError: x shape does not match training shape